<a href="https://colab.research.google.com/github/Loptt/forest-fire-detector/blob/main/classifier_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fire Classifier with autoencoders

## Data Loading

In [14]:
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras import Input
from keras.utils.vis_utils import plot_model


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

train_dir = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Original/Training"
test_dir = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Original/Test"

Mounted at /content/gdrive


In [57]:
datagen = ImageDataGenerator(rescale=1./255,
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True)

## Autoencoder creation

In [51]:
train_generator_autoencoder = datagen.flow_from_directory(train_dir,
                                                target_size=(254, 254), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='input')

test_generator_autoencoder = datagen.flow_from_directory(test_dir,
                                                target_size=(254, 254),
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='input')

Found 39401 images belonging to 2 classes.
Found 8646 images belonging to 2 classes.


Encoder

In [52]:
input_img = Input(shape=(254, 254, 3))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

Decoder

In [53]:
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (2, 2), activation='relu', padding='valid')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

Compile Model

In [59]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

plot_model(autoencoder, show_shapes=True, show_layer_names=True)
autoencoder.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 254, 254, 3)]     0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 127, 127, 16)      4624      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 64, 64, 16)        2320      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 32, 32, 16)        0  

In [61]:
autoencoder.fit(train_generator_autoencoder,
                epochs=50,
                batch_size=100,
                shuffle=True,
                validation_data=test_generator_autoencoder)

Epoch 1/50
  42/1971 [..............................] - ETA: 4:01:04 - loss: 0.6219

KeyboardInterrupt: ignored